In [163]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.cross_decomposition import CCA
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [164]:
print("Starting")
transactions = pd.read_csv('data/train_2016_v2.csv', parse_dates=['transactiondate'])
test_transactions = pd.read_csv('data/train_2017.csv', parse_dates=['transactiondate'])
properties = pd.read_csv('data/properties_2016.csv')
print("done")

Starting


/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


done


In [166]:
merged = pd.merge(transactions, properties, on="parcelid", how="left")
test_merged = pd.merge(test_transactions, properties, on="parcelid", how="left")

In [167]:
merged = merged.dropna(axis=1, how='all')
test_merged = merged.dropna(axis=1, how='all')

df = merged.filter(regex='id')
df.fillna(df.mode().iloc[0], axis=0, inplace=True)

test_df = test_merged.filter(regex='id')
test_df.fillna(test_df.mode().iloc[0], axis=0, inplace=True)

merged.fillna(merged.mean(), axis=0, inplace=True)
merged[list(df.keys())] = df

test_merged.fillna(test_merged.mean(), axis=0, inplace=True)
test_merged[list(test_df.keys())] = df

for f in merged.columns:
    if merged[f].dtype=='object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(merged[f].values)) 
        merged[f] = lbl.transform(list(merged[f].values))
        
for f in test_merged.columns:
    if test_merged[f].dtype=='object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(test_merged[f].values)) 
        test_merged[f] = lbl.transform(list(test_merged[f].values))

Y = merged.logerror.values
X = merged.drop(["parcelid", "transactiondate", "logerror"], axis=1)

Y_test = test_merged.logerror.values
X_test = test_merged.drop(["parcelid", "transactiondate", "logerror"], axis=1)

In [162]:
est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=20, random_state=0, loss='ls')
est.fit(X, Y)
est.score(X, Y)

0.94046595909819708

In [169]:
mean_squared_error(Y_test, est.predict(X_test))
# est.score(X_test, Y_test)

0.0015446764041706835